### In this chapter we created own deep learning framework  
каждый тензор получает два новых атрибута. 

**creators** — это список любых тензоров, использовавшихся для создания текущего тензора (по умолчанию имеет значение None).То есть если тензор z получается сложением двух других тензоров, x и y, атрибут creators тензора z будет содержать тензоры x и y

**creation_op** - опутствующий атрибут, который хранит операции, использовавшиеся в процессе создания данного тензора. *То есть инструкция z = x + y создаст вычислительный граф с тремя узлами (x, y и z) и двумя ребрами (z -> x и z -> y). Каждое ребро при этом подписано операцией из creation_op, то есть add.* Этот граф поможет организовать рекурсивное обратное распространение градиентов.


In [27]:
import numpy as np

class Tensor(object):
    
    
    def __init__(self, data, creators=None, creation_op=None):
        self.data = np.array(data)
        self.creators = creators
        self.creation_op = creation_op
        self.grad = None
    
    # Автоматическое вычисление градиента
    def backward(self, grad):
        self.grad = grad
        if (self.creation_op == 'add'):
            self.creators[0].backward(grad)
            self.creators[1].backward(grad)
        
    
    def __add__(self, other):
        return Tensor(self.data + other.data,
                        creators=[self, other],
                        creation_op='add')
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())

Tensor — возможность использовать граф для вычисления градиентов. **Если вызвать метод z.backward()**, он передаст градиент для x и y с учетом функции, с помощью которой создавался тензор z (add). Как показано в примере выше, мы передаем вектор градиентов (np.array([1,1,1,1,1])) в z, а тот применяет его к своим родителям. Самой замечательной особенностью этой формы автоматического вычисления градиента является то, что она работает рекурсивно — каждый вектор вызывает метод .backward() всех своих родителей из списка self.creator. 

In [28]:
x = Tensor([1, 2, 3, 4, 5])
y = Tensor([2, 2, 2, 2, 2])
z = x + y
z.backward(Tensor(np.array([1, 1, 1, 1, 1])))

# В данном случае у нас только один градиент для добавления в x and y
print(x.grad)
print(y.grad)
print(z.creators)
print(z.creation_op)

[1 1 1 1 1]
[1 1 1 1 1]
[array([1, 2, 3, 4, 5]), array([2, 2, 2, 2, 2])]
add


In [29]:
a = Tensor([1,2,3,4,5])
b = Tensor([2,2,2,2,2])
c = Tensor([5,4,3,2,1])

d = a + b
e = b + c
f = d + e
f.backward(Tensor(np.array([1,1,1,1,1])))

b.grad.data == np.array([2,2,2,2,2])

array([False, False, False, False, False])

# Много кратное использование тензоров
Выше реализация имеет ошибку, нам нужно, что в случае многократного использования тензоров,  при накапливание градиента она получала градиент от всех потомков.

* Дополнительно **мы создали счетчик self.children, подсчитывающий количество градиентов, полученных от каждого потомка в процессе обратного распространения.** Так мы исключаем возможность случайного повторного обратного распространения от одного и того же потомка (в этом случае возбуждается исключение).
*  **all_children_grads_accounted_for(). Эта функция проверяет, были ли получены градиенты от всех потомков в графе.**
* Обычно, когда метод .backward() вызывается промежуточной переменной в графе, она тут же вызывает метод .backward() своего родителя. Но так как некоторые переменные получают свои градиенты от нескольких родителей, каждая должна отложить вызов .backward() своего родителя, пока не получит свой окончательный градиент.

In [ ]:
class Tensor (object):
    
    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        
        self.data = np.array(data)
        self.autograd = autograd
        self.grad = None
        if(id is None):
            self.id = np.random.randint(0,100000)
        else:
            self.id = id
        
        self.creators = creators
        self.creation_op = creation_op
        self.children = {}
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children): # Скорректировать число потомков данного тезнора
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1
    
    # Проверка на получили ли тензоры градиенты от всех потомков
    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if(cnt != 0):
                return False
        return True        
        
    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
            if(grad is None):
                grad = FloatTensor(np.ones_like(self.data))
            # Проверка возможности обратного распространения или ожидания градиента
            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            if(self.grad is None):
                self.grad = grad # Накопление градиента от нескольких потомков
            else:
                self.grad += grad
            
            # grads must not have grads of their own
            assert grad.autograd == False
            
            # only continue backpropping if there's something to
            # backprop into and if all gradients (from children)
            # are accounted for override waiting for children if
            # "backprop" was called on this variable directly
            if(self.creators is not None and 
               (self.all_children_grads_accounted_for() or 
                grad_origin is None)):

                if(self.creation_op == "add"):
                    # Фактически начало обратного распространения
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                    
    def __add__(self, other):
        # Включаем автограф
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="add")
        return Tensor(self.data + other.data)

    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())  

In [26]:
a = Tensor([1,2,3,4,5], autograd=True)
b = Tensor([2,2,2,2,2], autograd=True)
c = Tensor([5,4,3,2,1], autograd=True)

d = a + b
e = b + c
f = d + e

f.backward(Tensor(np.array([1,1,1,1,1])))

print(b.grad.data == np.array([2,2,2,2,2]))


[ True  True  True  True  True]
